In [2]:
import pandas as pd
import torch
import tqdm
from tqdm.auto import tqdm
tqdm.pandas()
from collections import Counter
import numpy as np

In [2]:
from datasets import load_dataset
dataset = load_dataset("code_search_net", "python")
df_train = dataset['train'].to_pandas().reset_index(drop = True)
df_val = dataset['validation'].to_pandas().reset_index(drop = True)
df_test = dataset['test'].to_pandas().reset_index(drop = True)

/Users/runqian0120/opt/anaconda3/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Dataset Sizes

In [7]:
df_train['func_code_len'] = df_train['func_code_tokens'].progress_apply(lambda x : len(x))
df_val['func_code_len'] = df_val['func_code_tokens'].progress_apply(lambda x : len(x))
df_test['func_code_len'] = df_test['func_code_tokens'].progress_apply(lambda x : len(x))

df_train = df_train[df_train['func_code_len'] <= 500]
df_val = df_val[df_val['func_code_len'] <= 500]
df_test = df_test[df_test['func_code_len'] <= 500]

  0%|          | 0/412178 [00:00<?, ?it/s]

  0%|          | 0/23107 [00:00<?, ?it/s]

  0%|          | 0/22176 [00:00<?, ?it/s]

In [8]:
print(f"Train Size: {df_train.shape}")
print(f"Validation Size: {df_val.shape}")
print(f"Test Size: {df_test.shape}")

Train Size: (402906, 12)
Validation Size: (22478, 12)
Test Size: (21736, 12)


In [9]:
df_train.head()

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url,func_code_len
0,proycon/pynlpl,pynlpl/formats/folia.py,AbstractElement.addidsuffix,"def addidsuffix(self, idsuffix, recursive = Tr...",python,"def addidsuffix(self, idsuffix, recursive = Tr...","[def, addidsuffix, (, self, ,, idsuffix, ,, re...","Appends a suffix to this element's ID, and opt...","[Appends, a, suffix, to, this, element, s, ID,...",train,https://github.com/proycon/pynlpl/blob/7707f69...,44
1,proycon/pynlpl,pynlpl/formats/folia.py,AbstractElement.setparents,"def setparents(self):\n """"""Correct all ...",python,"def setparents(self):\n """"""Correct all ...","[def, setparents, (, self, ), :, for, c, in, s...",Correct all parent relations for elements with...,"[Correct, all, parent, relations, for, element...",train,https://github.com/proycon/pynlpl/blob/7707f69...,29
2,proycon/pynlpl,pynlpl/formats/folia.py,AbstractElement.setdoc,"def setdoc(self,newdoc):\n """"""Set a dif...",python,"def setdoc(self,newdoc):\n """"""Set a dif...","[def, setdoc, (, self, ,, newdoc, ), :, self, ...",Set a different document. Usually no need to c...,"[Set, a, different, document, ., Usually, no, ...",train,https://github.com/proycon/pynlpl/blob/7707f69...,53
3,proycon/pynlpl,pynlpl/formats/folia.py,AbstractElement.hastext,"def hastext(self,cls='current',strict=True, co...",python,"def hastext(self,cls='current',strict=True, co...","[def, hastext, (, self, ,, cls, =, 'current', ...",Does this element have text (of the specified ...,"[Does, this, element, have, text, (, of, the, ...",train,https://github.com/proycon/pynlpl/blob/7707f69...,106
4,proycon/pynlpl,pynlpl/formats/folia.py,AbstractElement.hasphon,"def hasphon(self,cls='current',strict=True,cor...",python,"def hasphon(self,cls='current',strict=True,cor...","[def, hasphon, (, self, ,, cls, =, 'current', ...",Does this element have phonetic content (of th...,"[Does, this, element, have, phonetic, content,...",train,https://github.com/proycon/pynlpl/blob/7707f69...,103


In [10]:
df_test.head()

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url,func_code_len
0,soimort/you-get,src/you_get/extractors/youtube.py,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...",python,"def get_vid_from_url(url):\n """"""Extract...","[def, get_vid_from_url, (, url, ), :, return, ...",Extracts video ID from URL.,"[Extracts, video, ID, from, URL, .]",test,https://github.com/soimort/you-get/blob/b746ac...,53
1,soimort/you-get,src/you_get/extractors/miomio.py,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...",python,"def sina_xml_to_url_list(xml_data):\n """"""st...","[def, sina_xml_to_url_list, (, xml_data, ), :,...",str->list\n Convert XML to URL List.\n F...,"[str, -, >, list, Convert, XML, to, URL, List,...",test,https://github.com/soimort/you-get/blob/b746ac...,52
2,soimort/you-get,src/you_get/extractors/fc2video.py,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....",python,"def makeMimi(upid):\n """"""From http://cdn37....","[def, makeMimi, (, upid, ), :, strSeed, =, ""gG...",From http://cdn37.atwikiimg.com/sitescript/pub...,"[From, http, :, //, cdn37, ., atwikiimg, ., co...",test,https://github.com/soimort/you-get/blob/b746ac...,30
3,soimort/you-get,src/you_get/extractors/fc2video.py,fc2video_download,"def fc2video_download(url, output_dir = '.', m...",python,"def fc2video_download(url, output_dir = '.', m...","[def, fc2video_download, (, url, ,, output_dir...",wrapper,[wrapper],test,https://github.com/soimort/you-get/blob/b746ac...,66
4,soimort/you-get,src/you_get/extractors/dailymotion.py,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...",python,"def dailymotion_download(url, output_dir='.', ...","[def, dailymotion_download, (, url, ,, output_...",Downloads Dailymotion videos by URL.,"[Downloads, Dailymotion, videos, by, URL, .]",test,https://github.com/soimort/you-get/blob/b746ac...,150


In [11]:
df_val.head()

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url,func_code_len
1,openai/baselines,baselines/deepq/deepq.py,ActWrapper.save_act,"def save_act(self, path=None):\n """"""Sav...",python,"def save_act(self, path=None):\n """"""Sav...","[def, save_act, (, self, ,, path, =, None, ), ...",Save model to a pickle located at `path`,"[Save, model, to, a, pickle, located, at, path]",valid,https://github.com/openai/baselines/blob/33010...,171
2,openai/baselines,baselines/common/models.py,nature_cnn,"def nature_cnn(unscaled_images, **conv_kwargs)...",python,"def nature_cnn(unscaled_images, **conv_kwargs)...","[def, nature_cnn, (, unscaled_images, ,, *, *,...",CNN from Nature paper.,"[CNN, from, Nature, paper, .]",valid,https://github.com/openai/baselines/blob/33010...,168
3,openai/baselines,baselines/common/models.py,mlp,"def mlp(num_layers=2, num_hidden=64, activatio...",python,"def mlp(num_layers=2, num_hidden=64, activatio...","[def, mlp, (, num_layers, =, 2, ,, num_hidden,...",Stack of fully-connected layers to be used in ...,"[Stack, of, fully, -, connected, layers, to, b...",valid,https://github.com/openai/baselines/blob/33010...,105
4,openai/baselines,baselines/common/models.py,lstm,"def lstm(nlstm=128, layer_norm=False):\n """"...",python,"def lstm(nlstm=128, layer_norm=False):\n """"...","[def, lstm, (, nlstm, =, 128, ,, layer_norm, =...",Builds LSTM (Long-Short Term Memory) network t...,"[Builds, LSTM, (, Long, -, Short, Term, Memory...",valid,https://github.com/openai/baselines/blob/33010...,194
5,openai/baselines,baselines/common/models.py,conv_only,"def conv_only(convs=[(32, 8, 4), (64, 4, 2), (...",python,"def conv_only(convs=[(32, 8, 4), (64, 4, 2), (...","[def, conv_only, (, convs, =, [, (, 32, ,, 8, ...",convolutions-only net\n\n Parameters:\n ...,"[convolutions, -, only, net]",valid,https://github.com/openai/baselines/blob/33010...,109


## Getting AST of functions

In [12]:
df_train = df_train.head(100000)
df_val = df_val.head(5000)
df_test = df_test.head(5000)

In [13]:
import ast


## Filter out AST Errors (caused by comments in foreign languages)

In [16]:
def get_ast(code_str):
    try:
        return ast.dump(ast.parse(code_str), indent=2)
    except SyntaxError as e:
        return ''
    
df_train['ast'] = df_train['func_code_string'].progress_apply(get_ast)
df_val['ast'] = df_val['func_code_string'].progress_apply(get_ast)
df_test['ast'] = df_test['func_code_string'].progress_apply(get_ast)

df_train = df_train[df_train['ast'] != '']#.iloc[0]['whole_func_string']
df_val = df_val[df_val['ast'] != '']
df_test = df_test[df_test['ast'] != '']

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [17]:
def get_tokenized_ast(code_str):
    def tokenize_ast(node):
        tokens = []
        for child in ast.iter_child_nodes(node):
            # Tokenize the node type
            tokens.append(type(child).__name__)
            # Recursively tokenize children
            tokens.extend(tokenize_ast(child))
        return tokens

    # Tokenize the AST starting from the root
    root = ast.parse(code_str)
    return tokenize_ast(root)

df_train['ast_tokens'] = df_train['func_code_string'].progress_apply(get_tokenized_ast)
df_val['ast_tokens'] = df_val['func_code_string'].progress_apply(get_tokenized_ast)
df_test['ast_tokens'] = df_test['func_code_string'].progress_apply(get_tokenized_ast)

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

In [81]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(98803, 17)
(4880, 17)
(4983, 17)


## Adding Doc Strings

#### Enrich with helpful information for docstring

In [72]:
def add_params(df):
    def calc_num_params(tokens) :
        try:
            close_idx = np.where(tokens ==')')[0][0]
            open_idx = np.where(tokens=='(')[0][0]
            segment = tokens[open_idx + 1:close_idx]  # Slice the array to get the segment between '(' and ')'
            commas = np.count_nonzero(segment == ',')
            return commas+1
        except:
            # print(tokens)
            return -1

    df['fn_name'] = df['func_name'].apply(lambda x: x.split('.')[-1])
    df['class_fn'] = df['func_name'].apply(lambda x: '.' in x)
    df['num_params'] = df['func_code_tokens'].apply(calc_num_params)
    return df

df_train = add_params(df_train)
df_val = add_params(df_val)
df_test = add_params(df_test)

In [73]:
def calc_doc_string(df):
    dfs = []

    def check_if_function_used(tokens, fn_name):
        m1 = tokens[:-1] == fn_name
        m2 = tokens[1:] == '('
        combined_mask = m1 & m2

        indices = np.where(combined_mask)[0]
        if indices.size == 0:
            return -1
        return indices[0]

    all_repos = set(df['repository_name'])
    # print(df)
    # print(all_repos)

    for repo_name in tqdm(all_repos):
        filtered = df[df['repository_name'] == repo_name]
        filtered['used_function_docstring'] = filtered.apply(lambda row: {}, axis=1)

        # Get documentation string
        all_fns = set(filtered['fn_name'])
        for fn in all_fns:
            for i, row in filtered.iterrows():
                if row['fn_name'] == fn:
                    continue
                # print(fn)
                # print(row)
                try:
                    fn_body = row['func_code_tokens'][np.where(row['func_code_tokens'] == ':')[0][0]+1:]
                except:
                    continue
                
                index = check_if_function_used(fn_body, fn)

                if index != -1: 
                    # Check num params
                    if fn_body[index-1] == '.': 
                        # function called a class method
                        doc_string = filtered[(filtered['fn_name'] == fn) & (filtered['class_fn'])].reset_index(drop=True)
                        try:
                            filtered.loc[i, 'used_function_docstring'][fn] = doc_string.iloc[0]['func_documentation_tokens']
                        except:
                            pass
                    else :
                        # function called a non-class method
                        doc_string = filtered[(filtered['fn_name'] == fn) & (filtered['class_fn'] == False)].reset_index(drop=True)
                        try:
                            filtered.loc[i, 'used_function_docstring'][fn] = doc_string.iloc[0]['func_documentation_tokens']
                        except:
                            pass
        dfs.append(filtered)
    return pd.concat(dfs).reset_index(drop=True)

df_train = calc_doc_string(df_train)
df_val = calc_doc_string(df_val)
df_test = calc_doc_string(df_test)


  0%|          | 0/4154 [00:00<?, ?it/s]

/var/folders/2t/31twf06s5gq20q0f2m2h2zlh0000gn/T/ipykernel_13121/4234615731.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['used_function_docstring'] = filtered.apply(lambda row: {}, axis=1)


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

## Dropping not used columns

In [76]:
df_train = df_train.drop(columns = ['fn_name', 'class_fn', 'num_params'])
df_val = df_val.drop(columns = ['fn_name', 'class_fn', 'num_params'])
df_test = df_test.drop(columns = ['fn_name', 'class_fn', 'num_params'])

In [74]:
df_train.head(10)

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,...,ast_tokens,processed_func_code_tokens,processed_ast_code_tokens,processed_doc_tokens,built_in_fns,std_lib_fn,fn_name,class_fn,num_params,used_function_docstring
0,why2pac/dp-tornado,dp_tornado/helper/serialization/json.py,JsonHelper.stringify,"def stringify(self, obj, beautify=False, raise...",python,"def stringify(self, obj, beautify=False, raise...","[def, stringify, (, self, ,, obj, ,, beautify,...",Alias of helper.string.serialization.json.stri...,"[Alias, of, helper, ., string, ., serializatio...",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, C...","[<sos>, def, stringify, (, self, ,, obj, ,, be...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Alias, of, helper, ., string, ., seria...",{},{},stringify,True,4,{}
1,why2pac/dp-tornado,dp_tornado/helper/serialization/json.py,JsonHelper.parse,"def parse(self, text, encoding='utf8', raise_e...",python,"def parse(self, text, encoding='utf8', raise_e...","[def, parse, (, self, ,, text, ,, encoding, =,...",Alias of helper.string.serialization.json.parse,"[Alias, of, helper, ., string, ., serializatio...",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, C...","[<sos>, def, parse, (, self, ,, text, ,, encod...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Alias, of, helper, ., string, ., seria...",{},{},parse,True,4,{}
2,why2pac/dp-tornado,dp_tornado/helper/serialization/__init__.py,SerializationHelper.serialize,"def serialize(self, obj, method='json', beauti...",python,"def serialize(self, obj, method='json', beauti...","[def, serialize, (, self, ,, obj, ,, method, =...",Alias of helper.string.serialization.serialize,"[Alias, of, helper, ., string, ., serializatio...",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, a...","[<sos>, def, serialize, (, self, ,, obj, ,, me...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Alias, of, helper, ., string, ., seria...",{},{},serialize,True,5,{}
3,why2pac/dp-tornado,dp_tornado/helper/serialization/__init__.py,SerializationHelper.deserialize,"def deserialize(self, text, method='json', enc...",python,"def deserialize(self, text, method='json', enc...","[def, deserialize, (, self, ,, text, ,, method...",Alias of helper.string.serialization.deserialize,"[Alias, of, helper, ., string, ., serializatio...",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, a...","[<sos>, def, deserialize, (, self, ,, text, ,,...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Alias, of, helper, ., string, ., seria...",{},{},deserialize,True,5,{}
4,yougov/excuses,excuses/__init__.py,RandomExcuseGenerator.pmxbot_excuse,"def pmxbot_excuse(self, rest):\n ""Provi...",python,"def pmxbot_excuse(self, rest):\n ""Provi...","[def, pmxbot_excuse, (, self, ,, rest, ), :, a...",Provide a convenient excuse,"[Provide, a, convenient, excuse]",train,...,"[FunctionDef, arguments, arg, arg, Expr, Const...","[<sos>, def, pmxbot_excuse, (, self, ,, rest, ...","[<sos>, FunctionDef, arguments, arg, arg, Expr...","[<sos>, Provide, a, convenient, excuse, <eos>,...",{},{},pmxbot_excuse,True,2,{}
5,tmr232/rage,rage/value.py,parse_value,"def parse_value(named_reg_value):\n """"""\n ...",python,"def parse_value(named_reg_value):\n """"""\n ...","[def, parse_value, (, named_reg_value, ), :, n...",Convert the value returned from EnumValue to a...,"[Convert, the, value, returned, from, EnumValu...",train,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, parse_value, (, named_reg_value, ...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, Convert, the, value, returned, from, E...",{},{},parse_value,False,1,{}
6,tmr232/rage,rage/rage.py,require_editable,"def require_editable(f):\n """"""\n Makes s...",python,"def require_editable(f):\n """"""\n Makes s...","[def, require_editable, (, f, ), :, def, wrap

In [56]:
df_val.head(10)

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url,ast,fn_name,class_fn,num_params,used_function_docstring,ast_tokens
0,aleju/imgaug,imgaug/augmentables/kps.py,compute_geometric_median,"def compute_geometric_median(X, eps=1e-5):\n ...",python,"def compute_geometric_median(X, eps=1e-5):\n ...","[def, compute_geometric_median, (, X, ,, eps, ...",Estimate the geometric median of points in 2D....,"[Estimate, the, geometric, median, of, points,...",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,compute_geometric_median,False,2,{},"[FunctionDef, arguments, arg, arg, Constant, E..."
1,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.project,"def project(self, from_shape, to_shape):\n ...",python,"def project(self, from_shape, to_shape):\n ...","[def, project, (, self, ,, from_shape, ,, to_s...",Project the keypoint onto a new position on a ...,"[Project, the, keypoint, onto, a, new, positio...",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,project,True,3,"{'project_coords': ['Project', 'coordinates', ...","[FunctionDef, arguments, arg, arg, arg, Expr, ..."
2,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.shift,"def shift(self, x=0, y=0):\n """"""\n ...",python,"def shift(self, x=0, y=0):\n """"""\n ...","[def, shift, (, self, ,, x, =, 0, ,, y, =, 0, ...",Move the keypoint around on an image.\n\n ...,"[Move, the, keypoint, around, on, an, image, .]",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,shift,True,3,"{'deepcopy': ['Create', 'a', 'deep', 'copy', '...","[FunctionDef, arguments, arg, arg, arg, Consta..."
3,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.draw_on_image,"def draw_on_image(self, image, color=(0, 255, ...",python,"def draw_on_image(self, image, color=(0, 255, ...","[def, draw_on_image, (, self, ,, image, ,, col...",Draw the keypoint onto a given image.\n\n ...,"[Draw, the, keypoint, onto, a, given, image, .]",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,draw_on_image,True,5,"{'copy': ['Create', 'a', 'shallow', 'copy', 'o...","[FunctionDef, arguments, arg, arg, arg, arg, a..."
4,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.generate_similar_points_manhattan,"def generate_similar_points_manhattan(self, nb...",python,"def generate_similar_points_manhattan(self, nb...","[def, generate_similar_points_manhattan, (, se...",Generate nearby points to this keypoint based ...,"[Generate, nearby, points, to, this, keypoint,...",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,generate_similar_points_manhattan,True,4,"{'deepcopy': ['Create', 'a', 'deep', 'copy', '...","[FunctionDef, arguments, arg, arg, arg, arg, C..."
5,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.copy,"def copy(self, x=None, y=None):\n """"""\n...",python,"def copy(self, x=None, y=None):\n """"""\n...","[def, copy, (, self, ,, x, =, None, ,, y, =, N...",Create a shallow copy of the Keypoint object.\...,"[Create, a, shallow, copy, of, the, Keypoint, ...",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,copy,True,3,"{'deepcopy': ['Create', 'a', 'deep', 'copy', '...","[FunctionDef, arguments, arg, arg, arg, Consta..."
6,aleju/imgaug,imgaug/augmentables/kps.py,Keypoint.deepcopy,"def deepcopy(self, x=None, y=None):\n ""...",python,"def deepcopy(self, x=None, y=None):\n ""...","[def, deepcopy, (, self, ,, x, =, None, ,, y, ...",Create a deep copy of the Keypoint object.\n\n...,"[Create, a, deep, copy, of, the, Keypoint, obj...",valid,https://github.com/aleju/imgaug/blob/786be74aa...,Module(\n body=[\n FunctionDef(\n nam...,deepcopy,True,3,{},"[FunctionDef, arguments, arg, arg, arg, Consta..."
7,aleju/imgaug,imgaug/augmentables/k

In [110]:
df_test.head(10)

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,...,ast_tokens,processed_func_code_tokens,processed_ast_code_tokens,processed_doc_tokens,built_in_fns,std_lib_fn,used_function_docstring,doc_strings,doc_string_tokenized,processed_doc_string
0,Qiskit/qiskit-terra,qiskit/circuit/instruction.py,Instruction.params,"def params(self):\n """"""return instructi...",python,"def params(self):\n """"""return instructi...","[def, params, (, self, ), :, # if params alrea...",return instruction params,"[return, instruction, params]",test,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, params, (, self, ), :, # if param...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, return, instruction, params, <eos>, <p...",{},{},"{'extend': ['Append', 'rhs', 'to', 'self', 'if...",extend: Append rhs to self if self contains co...,"[ext, end, :, ĠApp, end, Ġrh, s, Ġto, Ġself, Ġ...","[<sos>, ext, end, :, ĠApp, end, Ġrh, s, Ġto, Ġ..."
1,Qiskit/qiskit-terra,qiskit/circuit/instruction.py,Instruction.assemble,"def assemble(self):\n """"""Assemble a Qas...",python,"def assemble(self):\n """"""Assemble a Qas...","[def, assemble, (, self, ), :, instruction, =,...",Assemble a QasmQobjInstruction,"[Assemble, a, QasmQobjInstruction]",test,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, assemble, (, self, ), :, instruct...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, Assemble, a, QasmQobjInstruction, <eos...",{'hasattr': 'Return whether the object has an ...,{},{},hasattr: Return whether the object has an attr...,"[has, attr, :, ĠReturn, Ġwhether, Ġthe, Ġobjec...","[<sos>, has, attr, :, ĠReturn, Ġwhether, Ġthe,..."
2,Qiskit/qiskit-terra,qiskit/circuit/instruction.py,Instruction.mirror,"def mirror(self):\n """"""For a composite ...",python,"def mirror(self):\n """"""For a composite ...","[def, mirror, (, self, ), :, if, not, self, .,...","For a composite instruction, reverse the order...","[For, a, composite, instruction, reverse, the,...",test,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, mirror, (, self, ), :, if, not, s...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, For, a, composite, instruction, revers...","{'reversed': 'reversed(sequence, /) | | Return...",{},"{'append': ['Args', ':', 'passes', '(', 'list'...","reversed: reversed(sequence, /) | | Return a r...","[re, vers, ed, :, Ġreversed, (, sequence, ,, Ġ...","[<sos>, re, vers, ed, :, Ġreversed, (, sequenc..."
3,Qiskit/qiskit-terra,qiskit/circuit/instruction.py,Instruction.inverse,"def inverse(self):\n """"""Invert this ins...",python,"def inverse(self):\n """"""Invert this ins...","[def, inverse, (, self, ), :, if, not, self, ....",Invert this instruction.\n\n If the ins...,"[Invert, this, instruction, .]",test,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, inverse, (, self, ), :, if, not, ...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, Invert, this, instruction, ., <eos>, <...","{'reversed': 'reversed(sequence, /) | | Return...",{},"{'append': ['Args', ':', 'passes', '(', 'list'...","reversed: reversed(sequence, /) | | Return a r...","[re, vers, ed, :, Ġreversed, (, sequence, ,, Ġ...","[<sos>, re, vers, ed, :, Ġreversed, (, sequenc..."
4,Qiskit/qiskit-terra,qiskit/circuit/instruction.py,Instruction.c_if,"def c_if(self, classical, val):\n """"""Ad...",python,"def c_if(self, classical, val):\n """"""Ad...","[def, c_if, (, self, ,, classical, ,, val, ), ...",Add classical control on register classical an...,"[Add, classical, control, on, register, classi...",test,...,"[FunctionDef, arguments, arg, arg, arg, Expr, ...","[<sos>, def, c_if, (, self, ,, classical, ,, v...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Add, classical, control, on, register,...",{'isinstance': 'Return whether an object is an...,{},{},isinst

## Pickle Dataframes for later use

In [111]:
df_train.to_pickle("df_train_doc_strings.pkl")
df_val.to_pickle("df_val_doc_strings.pkl")
df_test.to_pickle("df_test_doc_strings.pkl")

## Load Dataframes 

In [70]:
df_train = pd.read_pickle("df_train.pkl")
df_val = pd.read_pickle("df_val.pkl")
df_test = pd.read_pickle("df_test.pkl")

In [3]:
df_test.columns

Index(['repository_name', 'func_path_in_repository', 'func_name',
       'whole_func_string', 'language', 'func_code_string', 'func_code_tokens',
       'func_documentation_string', 'func_documentation_tokens', 'split_name',
       'func_code_url', 'func_code_len', 'ast', 'ast_tokens',
       'processed_func_code_tokens', 'processed_ast_code_tokens',
       'processed_doc_tokens'],
      dtype='object')

## Convert tokens to sequence of vectors

Metrics

In [20]:
mean_func_len = df_train['func_documentation_tokens'].apply(lambda x : len(x)).mean()
mean_ast_len = df_train['ast_tokens'].apply(lambda x : len(x)).mean()

print(mean_func_len)
print(mean_ast_len)

15.962480896329058
101.22493244132264


Padding + Truncate + sos + eos

In [5]:
def preprocess(toks, max_len):
    toks =np.insert(toks, 0, "<sos>")
    
    if len(toks) >= max_len : 
        toks = toks[:max_len-1]
        toks = np.append(toks,"<eos>")
    else:
        toks = np.append(toks,"<eos>")
        while len(toks) < max_len:
            toks = np.append(toks,'<pad>')
    return toks

df_train['processed_func_code_tokens'] = df_train['func_code_tokens'].progress_apply(lambda x: preprocess(x, 200))
df_val['processed_func_code_tokens'] = df_val['func_code_tokens'].progress_apply(lambda x: preprocess(x, 200))
df_test['processed_func_code_tokens'] = df_test['func_code_tokens'].progress_apply(lambda x: preprocess(x, 200))

df_train['processed_ast_code_tokens'] = df_train['ast_tokens'].progress_apply(lambda x: preprocess(x, 200))
df_val['processed_ast_code_tokens'] = df_val['ast_tokens'].progress_apply(lambda x: preprocess(x, 200))
df_test['processed_ast_code_tokens'] = df_test['ast_tokens'].progress_apply(lambda x: preprocess(x, 200))

df_train['processed_doc_tokens'] = df_train['func_documentation_tokens'].progress_apply(lambda x: preprocess(x, 100))
df_val['processed_doc_tokens'] = df_val['func_documentation_tokens'].progress_apply(lambda x: preprocess(x, 100))
df_test['processed_doc_tokens'] = df_test['func_documentation_tokens'].progress_apply(lambda x: preprocess(x, 100))

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

Training AST vectorization

In [25]:
# from gensim.models import Word2Vec

In [38]:
# training_asts = list(df_train['processed_ast_code_tokens'].progress_apply(lambda x: list(x)))
# ast_model = Word2Vec(sentences=training_asts, vector_size=64, window=20, min_count=1, workers=4)
# ast_model.save("ast_word2vec.model")

  0%|          | 0/98803 [00:00<?, ?it/s]

In [39]:
# training_func_codes = list(df_train['processed_func_code_tokens'].progress_apply(lambda x: list(x)))
# func_model = Word2Vec(sentences=training_func_codes, vector_size=64, window=20, min_count=1, workers=4)
# func_model.save("func_code_word2vec.model")

  0%|          | 0/98803 [00:00<?, ?it/s]

## Creating 1-hot Vectors

In [4]:
import torch
import torch.nn.functional as F
from collections import Counter

def calc_vocab(series, vocab_size): 
    tokenized_sentences = list(series.apply(lambda x: list(x)))
    all_words = [word for sentence in tokenized_sentences for word in sentence]

    word_freq = Counter(all_words)

    most_common_words = word_freq.most_common(vocab_size)

    vocabulary = {word: idx for idx, (word, freq) in enumerate(most_common_words)}

    unknown_token = '<unk>'
    vocabulary[unknown_token] = len(vocabulary)
    return vocabulary 

func_code_vocab = calc_vocab(df_train['processed_func_code_tokens'], 10000)
ast_code_vocab = calc_vocab(df_train['processed_ast_code_tokens'], 93)
doc_vocab = calc_vocab(df_train['processed_doc_tokens'], 10000)

In [5]:
def get_one_hot(series, vocabulary): 
    tokenized_sentences = list(series.apply(lambda x: list(x)))
    unknown_token = '<unk>'
    def sentence_to_indices(sentence, vocab):
        return [vocab[word] if word in vocab else vocab[unknown_token] for word in sentence]

    indices_sentences = [sentence_to_indices(sentence, vocabulary) for sentence in tokenized_sentences]
    num_classes = len(vocabulary)  
    one_hot_sentences = [F.one_hot(torch.tensor(sentence), num_classes=num_classes) for sentence in indices_sentences]
    return one_hot_sentences

func_code_one_hot = get_one_hot(df_train['processed_func_code_tokens'], func_code_vocab)
# ast_one_hot = get_one_hot(df_train['processed_ast_code_tokens'], ast_code_vocab)
# doc_one_hot = get_one_hot(df_train['processed_doc_tokens'], doc_vocab)

In [21]:
df_train_reduced = df_train[['processed_func_code_tokens', 'processed_ast_code_tokens', 'processed_doc_tokens']]
df_val_reduced = df_val[['processed_func_code_tokens', 'processed_ast_code_tokens', 'processed_doc_tokens']]
df_test_reduced = df_test[['processed_func_code_tokens', 'processed_ast_code_tokens', 'processed_doc_tokens']]

In [106]:
df_train_reduced.to_pickle("df_train_reduced.pkl")
df_val_reduced.to_pickle("df_val_reduced.pkl")
df_test_reduced.to_pickle("df_test_reduced.pkl")

NameError: name 'df_train_reduced' is not defined

## Getting Python StdLib

In [70]:
import pkgutil
import zipimport
import inspect
import io 
import contextlib
import re

def sanitize_help_string(s):
    splits = s.split('\n')
    cleaned = "\n".join(splits[3:]).strip()
    if len(cleaned) and cleaned[0] == '|':
        cleaned = cleaned[1:].strip()    
    return re.sub(r'\s+', ' ',cleaned)

def get_help_as_string(obj):
    output = io.StringIO()
    with contextlib.redirect_stdout(output):
        help(obj)
    s = sanitize_help_string(output.getvalue())

    if len(s.split("|")) > 1: 
        print("|".join(s.split("|")[:5]))
        return "|".join(s.split("|")[:5])
    return s

def get_std_lib_doc():
    std_modules = []
    
    for importer, modname, ispkg in pkgutil.iter_modules():
        importer_path = ''
        if hasattr(importer, 'path'):
            importer_path  = importer.path
        elif isinstance(importer, zipimport.zipimporter):
            importer_path = importer.archive

        if 'lib/python3.9' == importer_path[-13:]:
            # print(importer, modname, ispkg)
            module = __import__(modname)
            std_modules.append(module)

    stdlib_functions = {}
    for module in std_modules:
        module_functions = []
        for name, obj in inspect.getmembers(module):
            if '_' != name[0] and (inspect.isfunction(obj) or inspect.isbuiltin(obj)):
                module_functions.append((name, obj))
        if module_functions:
            stdlib_functions[module.__name__] = module_functions
    res = {}
    for k,v in stdlib_functions.items():
        res[k] = {}
        for f in v:    
            if f[0] != 'main':
                s = get_help_as_string(f[1])
                if s != '':
                    res[k][f[0]] = s
        if not len(res[k]) :
            res.pop(k)
            # print(f[0], help(f[1]))
    return res
std_lib_docs = get_std_lib_doc()

Sets the unittest option flags. The old flag is returned so that a runner could restore the old value if it wished to: >>> import doctest >>> old = doctest._unittest_reportflags >>> doctest.set_unittest_reportflags(REPORT_NDIFF | ... REPORT_ONLY_FIRST_FAILURE) == old True >>> doctest._unittest_reportflags == (REPORT_NDIFF | ... REPORT_ONLY_FIRST_FAILURE) True Only reporting flags can be set: >>> doctest.set_unittest_reportflags(ELLIPSIS) Traceback (most recent call last): ... ValueError: ('Only reporting flags allowed', 8) >>> doctest.set_unittest_reportflags(old) == (REPORT_NDIFF | ... REPORT_ONLY_FIRST_FAILURE) True
Parse the '229' response for an EPSV request. Raises error_proto if it does not contain '(|||port|)' Return ('host.addr.as.numbers', port#) tuple.
Return true if the object is a code object. Code objects provide these attributes: co_argcount number of arguments (not including *, ** args or keyword only arguments) co_code string of raw compiled bytecode co_cellvars tuple o

In [82]:
built_in_functions_dict = {
    'abs': abs, 'all': all, 'any': any, 'ascii': ascii, 'bin': bin, 'bool': bool,
    'bytearray': bytearray, 'bytes': bytes, 'callable': callable, 'chr': chr,
    'classmethod': classmethod, 'compile': compile, 'complex': complex,
    'delattr': delattr, 'dict': dict, 'dir': dir, 'divmod': divmod,
    'enumerate': enumerate, 'eval': eval, 'exec': exec, 'filter': filter,
    'float': float, 'format': format, 'frozenset': frozenset, 'getattr': getattr,
    'globals': globals, 'hasattr': hasattr, 'hash': hash, 'help': help, 'hex': hex,
    'id': id, 'input': input, 'int': int, 'isinstance': isinstance,
    'issubclass': issubclass, 'iter': iter, 'len': len, 'list': list,
    'locals': locals, 'map': map, 'max': max, 'memoryview': memoryview,
    'min': min, 'next': next, 'object': object, 'oct': oct, 'open': open, 'ord': ord,
    'pow': pow, 'print': print, 'property': property, 'range': range, 'repr': repr,
    'reversed': reversed, 'round': round, 'set': set, 'setattr': setattr, 'slice': slice,
    'sorted': sorted, 'staticmethod': staticmethod, 'str': str, 'sum': sum, 'super': super,
    'tuple': tuple, 'type': type, 'vars': vars, 'zip': zip, '__import__': __import__
}
built_in_functions_docs = {k:get_help_as_string(v) for k,v in built_in_functions_dict.items()}
built_in_functions_docs


bool(x) -> bool | | Returns True when the argument x is true, False otherwise. | The builtins True and False are the only two instances of the class bool. | The class bool is a subclass of the class int, and cannot be subclassed. 
bytearray(iterable_of_ints) -> bytearray | bytearray(string, encoding[, errors]) -> bytearray | bytearray(bytes_or_buffer) -> mutable copy of bytes_or_buffer | bytearray(int) -> bytes array of size given by the parameter initialized with null bytes | bytearray() -> empty bytes array 
bytes(iterable_of_ints) -> bytes | bytes(string, encoding[, errors]) -> bytes | bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer | bytes(int) -> bytes object of size given by the parameter initialized with null bytes | bytes() -> empty bytes object 
classmethod(function) -> method | | Convert a function to be a class method. | | A class method receives the class as implicit first argument, 
complex(real=0, imag=0) | | Create a complex number from a real part and an opt

{'abs': 'Return the absolute value of the argument.',
 'all': 'Return True if bool(x) is True for all values x in the iterable. If the iterable is empty, return True.',
 'any': 'Return True if bool(x) is True for any x in the iterable. If the iterable is empty, return False.',
 'ascii': 'Return an ASCII-only representation of an object. As repr(), return a string containing a printable representation of an object, but escape the non-ASCII characters in the string returned by repr() using \\\\x, \\\\u or \\\\U escapes. This generates a string similar to that returned by repr() in Python 2.',
 'bin': "Return the binary representation of an integer. >>> bin(2796202) '0b1010101010101010101010'",
 'bool': 'bool(x) -> bool | | Returns True when the argument x is true, False otherwise. | The builtins True and False are the only two instances of the class bool. | The class bool is a subclass of the class int, and cannot be subclassed. ',
 'bytearray': 'bytearray(iterable_of_ints) -> bytearray 

In [81]:
# std_lib_docs['built-in'] = built_in_functions_docs
std_lib_docs

{'_aix_support': {'aix_buildtag': 'Return the platform_tag of the system Python was built on.',
  'aix_platform': 'AIX filesets are identified by four decimal values: V.R.M.F. V (version) and R (release) can be retreived using ``uname`` Since 2007, starting with AIX 5.3 TL7, the M value has been included with the fileset bos.rte and represents the Technology Level (TL) of AIX. The F (Fix) value also increases, but is not relevant for comparing releases and binary compatibility. For binary compatibility the so-called builddate is needed. Again, the builddate of an AIX release is associated with bos.rte. AIX ABI compatibility is described as guaranteed at: https://www.ibm.com/ support/knowledgecenter/en/ssw_aix_72/install/binary_compatability.html For pep425 purposes the AIX platform tag becomes: "aix-{:1x}{:1d}{:02d}-{:04d}-{}".format(v, r, tl, builddate, bitsize) e.g., "aix-6107-1415-32" for AIX 6.1 TL7 bd 1415, 32-bit and, "aix-6107-1415-64" for AIX 6.1 TL7 bd 1415, 64-bit'},
 '_boots

## Matching Builtin Functions

In [83]:
def check_containment_builtin(fn_body, fn):
    for i in range(1,len(fn_body)-1):
        if fn_body[i] == fn and fn_body[i-1] != '.' and fn_body[i+1] == '(':
            return True 
    return False

def get_used_builtin(x):
    try:
        fn_body = x[np.where(x == ':')[0][0]+1:]
    except:
        return {}
    
    d = {}
    for k,v in built_in_functions_docs.items():
        if check_containment_builtin(fn_body,k): 
            d[k] = v
    return d

df_train['built_in_fns'] = df_train['func_code_tokens'].progress_apply(lambda x : get_used_builtin(x))
df_test['built_in_fns'] = df_test['func_code_tokens'].progress_apply(lambda x : get_used_builtin(x))
df_val['built_in_fns'] = df_val['func_code_tokens'].progress_apply(lambda x : get_used_builtin(x))

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

In [84]:
def check_containment_std(fn_body, fn, module):
    for i in range(0,len(fn_body)-3):
        if fn_body[i] == module and fn_body[i+1] != '.' and fn_body[i+2] == fn and fn_body[i+3] == '(':
            return True 
    return False

def get_used_std_lib(x):
    d = {}
    try:
        fn_body = x[np.where(x == ':')[0][0]+1:]
    except:
        return {}

    for k,v in std_lib_docs.items():
        for k1, v1 in v.items():
            if check_containment_std(fn_body, k1, k): 
                d[k + "." + k1] = v1
    return d

df_train['std_lib_fn'] = df_train['func_code_tokens'].progress_apply(lambda x : get_used_std_lib(x))
df_test['std_lib_fn'] = df_test['func_code_tokens'].progress_apply(lambda x : get_used_std_lib(x))
df_val['std_lib_fn'] = df_val['func_code_tokens'].progress_apply(lambda x : get_used_std_lib(x))

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

In [67]:
for i,row in df_val.iterrows():
    if len(row['built_in_fns']) :
        print(i)

1
3
6
9
10
11
13
15
17
18
20
22
24
25
27
29
31
32
33
35
36
38
39
40
41
45
47
48
49
51
52
53
54
55
57
58
62
63
64
65
66
70
71
72
74
75
76
80
81
82
83
84
86
90
93
94
96
98
101
102
108
110
111
112
113
115
116
119
120
127
128
134
139
142
143
144
147
148
154
155
156
157
158
162
163
166
167
168
169
172
174
176
178
181
183
189
190
191
192
193
198
199
204
206
209
210
213
214
215
218
220
221
222
223
231
232
233
234
235
236
237
238
239
244
245
247
253
260
267
268
269
270
271
272
275
276
277
279
282
283
288
289
290
292
296
308
309
314
316
319
321
327
342
344
349
350
361
362
365
369
373
380
382
383
384
387
388
392
393
394
399
412
415
417
418
425
435
438
440
441
451
452
455
458
461
466
467
472
476
477
480
481
483
488
491
492
497
498
503
506
509
514
515
516
517
518
519
521
522
523
526
528
529
530
531
533
536
537
539
540
543
544
545
546
551
553
554
560
561
562
568
573
580
581
583
588
590
591
592
593
594
596
597
601
602
603
604
605
606
607
608
612
616
619
632
633
634
637
638
641
642
650
651
652
653
65

In [75]:
df_train = pd.read_pickle('df_train_doc_strings.pkl')
df_test = pd.read_pickle('df_test_doc_strings.pkl')
df_val = pd.read_pickle('df_val_doc_strings.pkl')

KeyboardInterrupt: 

In [92]:
df_train.iloc[98799]['doc_strings']

'isinstance: Return whether an object is an instance of a class or of a subclass thereof. A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B) or ...`` etc'

In [88]:
# Going to use max 3 docstrings
def combine_doc_strings(builtin, std, within_repo):
    used = 0
    res = ""

    for k,v in builtin.items():
        if used >= 3:
            break
        res += f"{k}: {v}\n"
        used += 1

    for k,v in std.items():
        if used >= 3:
            break
        res += f"{k}: {v}\n"
        used += 1

    for k,v in within_repo.items():
        if used >= 3:
            break
        doc = " ".join(v)
        res += f"{k}: {doc}\n"
        used += 1

    if res == "": return "This function does not use any external functions"
    else: return res.strip(" .\n")

df_test['doc_strings'] = df_test.progress_apply(lambda row: combine_doc_strings(row['built_in_fns'], row['std_lib_fn'], row['used_function_docstring']), axis =1 )
df_train['doc_strings'] = df_train.progress_apply(lambda row: combine_doc_strings(row['built_in_fns'], row['std_lib_fn'], row['used_function_docstring']), axis =1 )
df_val['doc_strings'] = df_val.progress_apply(lambda row: combine_doc_strings(row['built_in_fns'], row['std_lib_fn'], row['used_function_docstring']), axis =1 )

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

In [93]:
from transformers import RobertaTokenizer

# Load the tokenizer for CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")


In [94]:
df_train['doc_string_tokenized'] = df_train['doc_strings'].progress_apply(lambda x : tokenizer.tokenize(x))
df_test['doc_string_tokenized'] = df_test['doc_strings'].progress_apply(lambda x : tokenizer.tokenize(x))
df_val['doc_string_tokenized'] = df_val['doc_strings'].progress_apply(lambda x : tokenizer.tokenize(x))

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

In [99]:
df_train['doc_string_tokenized'].progress_apply(lambda x: len(x)).mean()

  0%|          | 0/98803 [00:00<?, ?it/s]

89.4071232654879

In [101]:
def preprocess(toks, max_len):
    toks =np.insert(toks, 0, "<sos>")
    
    if len(toks) >= max_len : 
        toks = toks[:max_len-1]
        toks = np.append(toks,"<eos>")
    else:
        toks = np.append(toks,"<eos>")
        while len(toks) < max_len:
            toks = np.append(toks,'<pad>')
    return toks

df_train['processed_doc_string'] = df_train['doc_string_tokenized'].progress_apply(lambda x: preprocess(x, 50))
df_test['processed_doc_string'] = df_test['doc_string_tokenized'].progress_apply(lambda x: preprocess(x, 50))
df_val['processed_doc_string'] = df_val['doc_string_tokenized'].progress_apply(lambda x: preprocess(x, 50))

  0%|          | 0/98803 [00:00<?, ?it/s]

  0%|          | 0/4983 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

In [105]:
print(df_train['processed_doc_string'].iloc[98799])
df_train['doc_strings'].iloc[98799]

['<sos>' 'is' 'instance' ':' 'ĠReturn' 'Ġwhether' 'Ġan' 'Ġobject' 'Ġis'
 'Ġan' 'Ġinstance' 'Ġof' 'Ġa' 'Ġclass' 'Ġor' 'Ġof' 'Ġa' 'Ġsubclass'
 'Ġthereof' '.' 'ĠA' 'Ġtuple' ',' 'Ġas' 'Ġin' 'Ġ``' 'is' 'instance' '('
 'x' ',' 'Ġ(' 'A' ',' 'ĠB' ',' 'Ġ...' '))' '``' ',' 'Ġmay' 'Ġbe' 'Ġgiven'
 'Ġas' 'Ġthe' 'Ġtarget' 'Ġto' 'Ġcheck' 'Ġagainst' '<eos>']


'isinstance: Return whether an object is an instance of a class or of a subclass thereof. A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B) or ...`` etc'

In [136]:
df_val

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,...,ast_tokens,processed_func_code_tokens,processed_ast_code_tokens,processed_doc_tokens,built_in_fns,std_lib_fn,used_function_docstring,doc_strings,doc_string_tokenized,processed_doc_string
0,encode/starlette,starlette/datastructures.py,MutableHeaders.setdefault,"def setdefault(self, key: str, value: str) -> ...",python,"def setdefault(self, key: str, value: str) -> ...","[def, setdefault, (, self, ,, key, :, str, ,, ...","If the header `key` does not exist, then set i...","[If, the, header, key, does, not, exist, then,...",valid,...,"[FunctionDef, arguments, arg, arg, Name, Load,...","[<sos>, def, setdefault, (, self, ,, key, :, s...","[<sos>, FunctionDef, arguments, arg, arg, Name...","[<sos>, If, the, header, key, does, not, exist...","{'enumerate': 'enumerate(iterable, start=0) | ...",{},"{'append': ['Append', 'a', 'header', 'preservi...","enumerate: enumerate(iterable, start=0) | | Re...","[en, umer, ate, :, Ġenumer, ate, (, iter, able...","[<sos>, en, umer, ate, :, Ġenumer, ate, (, ite..."
1,encode/starlette,starlette/datastructures.py,MutableHeaders.append,"def append(self, key: str, value: str) -> None...",python,"def append(self, key: str, value: str) -> None...","[def, append, (, self, ,, key, :, str, ,, valu...","Append a header, preserving any duplicate entr...","[Append, a, header, preserving, any, duplicate...",valid,...,"[FunctionDef, arguments, arg, arg, Name, Load,...","[<sos>, def, append, (, self, ,, key, :, str, ...","[<sos>, FunctionDef, arguments, arg, arg, Name...","[<sos>, Append, a, header, preserving, any, du...",{},{},{},This function does not use any external functions,"[This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġany, Ġex...","[<sos>, This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġa..."
2,encode/starlette,starlette/routing.py,request_response,def request_response(func: typing.Callable) ->...,python,def request_response(func: typing.Callable) ->...,"[def, request_response, (, func, :, typing, .,...",Takes a function or coroutine `func(request) -...,"[Takes, a, function, or, coroutine, func, (, r...",valid,...,"[FunctionDef, arguments, arg, Attribute, Name,...","[<sos>, def, request_response, (, func, :, typ...","[<sos>, FunctionDef, arguments, arg, Attribute...","[<sos>, Takes, a, function, or, coroutine, fun...",{},{},{},This function does not use any external functions,"[This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġany, Ġex...","[<sos>, This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġa..."
3,encode/starlette,starlette/routing.py,websocket_session,def websocket_session(func: typing.Callable) -...,python,def websocket_session(func: typing.Callable) -...,"[def, websocket_session, (, func, :, typing, ....","Takes a coroutine `func(session)`, and returns...","[Takes, a, coroutine, func, (, session, ), and...",valid,...,"[FunctionDef, arguments, arg, Attribute, Name,...","[<sos>, def, websocket_session, (, func, :, ty...","[<sos>, FunctionDef, arguments, arg, Attribute...","[<sos>, Takes, a, coroutine, func, (, session,...",{},{},{},This function does not use any external functions,"[This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġany, Ġex...","[<sos>, This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġa..."
4,encode/starlette,starlette/routing.py,compile_path,def compile_path(\n path: str\n) -> typing....,python,def compile_path(\n path: str\n) -> typing....,"[def, compile_path, (, path, :, str, ), ->, ty...","Given a path string, like: ""/{username:str}"", ...","[Given, a, path, string, like, :, /, {, userna...",valid,...,"[FunctionDef, arguments, arg, Name, Load, Expr...","[<sos>, def, compile_path, (, path, :, str, ),...","[<sos>, FunctionDef, arguments, arg, Name, Loa...","[<sos>, Given, a, path, string, like, :, /, {,...",{},{},{},This function does not use any external functions,"[This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġany, Ġex...","[<sos>, This, Ġfunction, Ġdoes, Ġnot, Ġuse, Ġa..."
..

In [122]:
df_train[df_train['doc_strings'].str.contains('')]

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,...,ast_tokens,processed_func_code_tokens,processed_ast_code_tokens,processed_doc_tokens,built_in_fns,std_lib_fn,used_function_docstring,doc_strings,doc_string_tokenized,processed_doc_string
7,fangpenlin/pyramid-handy,pyramid_handy/tweens/allow_origin.py,allow_origin_tween_factory,"def allow_origin_tween_factory(handler, regist...",python,"def allow_origin_tween_factory(handler, regist...","[def, allow_origin_tween_factory, (, handler, ...",Allow cross origin XHR requests,"[Allow, cross, origin, XHR, requests]",train,...,"[FunctionDef, arguments, arg, arg, Expr, Const...","[<sos>, def, allow_origin_tween_factory, (, ha...","[<sos>, FunctionDef, arguments, arg, arg, Expr...","[<sos>, Allow, cross, origin, XHR, requests, <...",{'callable': 'Return whether the object is cal...,{},{},callable: Return whether the object is callabl...,"[call, able, :, ĠReturn, Ġwhether, Ġthe, Ġobje...","[<sos>, call, able, :, ĠReturn, Ġwhether, Ġthe..."
12,frascoweb/frasco-users,frasco_users/__init__.py,UsersFeature.current,"def current(self):\n """"""Returns the cur...",python,"def current(self):\n """"""Returns the cur...","[def, current, (, self, ), :, if, not, has_req...",Returns the current user,"[Returns, the, current, user]",train,...,"[FunctionDef, arguments, arg, Expr, Constant, ...","[<sos>, def, current, (, self, ), :, if, not, ...","[<sos>, FunctionDef, arguments, arg, Expr, Con...","[<sos>, Returns, the, current, user, <eos>, <p...","{'getattr': 'getattr(object, name[, default]) ...",{},{},"getattr: getattr(object, name[, default]) -> v...","[get, attr, :, Ġget, attr, (, object, ,, Ġname...","[<sos>, get, attr, :, Ġget, attr, (, object, ,..."
15,frascoweb/frasco-users,frasco_users/__init__.py,UsersFeature.update_password,"def update_password(self, user, password, skip...",python,"def update_password(self, user, password, skip...","[def, update_password, (, self, ,, user, ,, pa...",Updates the password of a user,"[Updates, the, password, of, a, user]",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, C...","[<sos>, def, update_password, (, self, ,, user...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Updates, the, password, of, a, user, <...","{'getattr': 'getattr(object, name[, default]) ...",{},{},"getattr: getattr(object, name[, default]) -> v...","[get, attr, :, Ġget, attr, (, object, ,, Ġname...","[<sos>, get, attr, :, Ġget, attr, (, object, ,..."
19,frascoweb/frasco-users,frasco_users/__init__.py,UsersFeature.post_signup,"def post_signup(self, user, login_user=None, s...",python,"def post_signup(self, user, login_user=None, s...","[def, post_signup, (, self, ,, user, ,, login_...",Executes post signup actions: sending the sign...,"[Executes, post, signup, actions, :, sending, ...",train,...,"[FunctionDef, arguments, arg, arg, arg, arg, C...","[<sos>, def, post_signup, (, self, ,, user, ,,...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Executes, post, signup, actions, :, se...","{'getattr': 'getattr(object, name[, default]) ...",{},"{'_login': ['Updates', 'user', 'attributes', '...","getattr: getattr(object, name[, default]) -> v...","[get, attr, :, Ġget, attr, (, object, ,, Ġname...","[<sos>, get, attr, :, Ġget, attr, (, object, ,..."
20,frascoweb/frasco-users,frasco_users/__init__.py,UsersFeature.gen_reset_password_token,"def gen_reset_password_token(self, user=None, ...",python,"def gen_reset_password_token(self, user=None, ...","[def, gen_reset_password_token, (, self, ,, us...",Generates a reset password token and optionnal...,"[Generates, a, reset, password, token, and, op...",train,...,"[FunctionDef, arguments, arg, arg, arg, Consta...","[<sos>, def, gen_reset_password_token, (, self...","[<sos>, FunctionDef, arguments, arg, arg, arg,...","[<sos>, Generates, a, reset, password, token, ...","{'getattr': 'getattr(ob

In [133]:
print(df_train.iloc[12]['func_documentation_string'])

Returns the current user


In [ ]:
df_train[''].progress_apply(lamda)

In [134]:
print(df_train.iloc[12]['doc_strings'])

getattr: getattr(object, name[, default]) -> value Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y. When a default argument is given, it is returned when the attribute doesn't exist; without it, an exception is raised in that case


In [128]:
pip install graphviz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.9 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [135]:
import ast
import graphviz

def visualize_ast(node, graph=None, parent=None, name=None):
    if graph is None:
        graph = graphviz.Digraph('AST', node_attr={'shape': 'box'})
        # Start the graph from the root node
        graph.node(str(id(node)), str(type(node).__name__))
    
    if parent is not None:
        # Connect current node with its parent
        graph.edge(str(id(parent)), str(id(node)), label=name)

    # Process all attributes that are nodes and all items in lists that are nodes
    for field, value in ast.iter_fields(node):
        if isinstance(value, ast.AST):
            graph.node(str(id(value)), str(type(value).__name__))
            graph.edge(str(id(node)), str(id(value)), label=field)
            visualize_ast(value, graph, node, field)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    graph.node(str(id(item)), str(type(item).__name__))
                    graph.edge(str(id(node)), str(id(item)), label=field)
                    visualize_ast(item, graph, node, field)

    return graph

parsed_ast = ast.parse(df_train.iloc[12]['whole_func_string'])
ast_graph = visualize_ast(parsed_ast)
ast_graph.render('ast', format='png', view=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'ast.png'

In [ ]:
df_train.to_pickle('df_train_v2.pkl')